In [6]:
import numpy as np
from sklearn.model_selection import StratifiedKFold

### KFolds

In [7]:
#===============================#
#==========PAN-STARRS===========#
#===============================#

train = np.load("datasets\ps1_train_48.npz", allow_pickle=True)
val = np.load("datasets\ps1_val_48.npz", allow_pickle=True)
test = np.load("datasets\ps1_test_48.npz", allow_pickle=True)


In [ ]:
#===============================#
#===============SDSS============#
#===============================#

train = np.load("datasets\h2f_multiresolution_train_n5.npz", allow_pickle=True)
val = np.load("datasets\h2f_multiresolution_val_n5.npz", allow_pickle=True)
test = np.load("datasets\h2f_multiresolution_test_n5.npz", allow_pickle=True)

In [8]:
range_z = np.linspace(0, 0.4, 181)[:-1]

z_train_class = np.array(np.digitize(train["metadata"].item()["z"].values,range_z)-1)

del train

z_val_class = np.array(np.digitize(val["metadata"].item()["z"].values,range_z)-1)

del val

z_test_class = np.array(np.digitize(test["metadata"].item()["z"].values,range_z)-1)

del test

In [9]:
full_z_class = np.concatenate([z_train_class, z_val_class,z_test_class])
full_idxs = np.arange(len(full_z_class))

In [9]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
skf.get_n_splits(full_idxs,full_z_class)

train_idxs = []
test_idxs =[]

for i, (train_index, test_index) in enumerate(skf.split(full_idxs, full_z_class)):
    train_idxs.append(train_index)
    test_idxs.append(test_index)

    #print(f"Fold {i}:")
    #print(f"  Train: index={train_index}")
    #print(f"  Test:  index={test_index}")

c:\Users\Legers\kfolds_splits\env_kfolds\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [10]:
dictado ={}

dictado["train_idxs"]  =train_idxs
dictado["test_idxs"]  =test_idxs

np.save("last_kfolds_idxs_ps1.npy", dictado)

### KFolds de epocas

In [10]:
folds = np.load("last_kfolds_idxs_ps1.npy",allow_pickle=True).item()

In [11]:
idx_train_1fold = folds["train_idxs"][0]

In [12]:
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
skf.get_n_splits(full_idxs[idx_train_1fold],full_z_class[idx_train_1fold])


train_idxs_sub = []
test_idxs_sub =[]

for i, (train_index, test_index) in enumerate(skf.split(full_idxs[idx_train_1fold], full_z_class[idx_train_1fold])):
    train_idxs_sub.append(full_idxs[idx_train_1fold][train_index])
    test_idxs_sub.append(full_idxs[idx_train_1fold][test_index])

    #print(f"Fold {i}:")
    #print(f"  Train: index={train_index}")
    #print(f"  Test:  index={test_index}")

c:\Users\Legers\kfolds_splits\env_kfolds\lib\site-packages\sklearn\model_selection\_split.py:776: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [13]:
dictado ={}

dictado["train_idxs"]  =train_idxs_sub
dictado["test_idxs"]  =test_idxs_sub

np.save("first_kfolds_idxs_ps1.npy", dictado)

### Generacion de datasets

In [1]:
import numpy as np
import pandas as pd

last_folds = np.load("last_kfolds_idxs.npy",allow_pickle=True).item()
#first_folds = np.load("first_kfolds_idxs.npy",allow_pickle=True).item()

#last_folds = np.load("last_kfolds_idxs_ps1.npy",allow_pickle=True).item()
#first_folds = np.load("first_kfolds_idxs_ps1.npy",allow_pickle=True).item()

In [2]:
def save_dataset(name_dataset,nlevels, unique=False):
    
    data = np.load(f"datasets\{name_dataset}.npz", allow_pickle=True)
    imgs_data = data["imgs"]
    meta_data = data["metadata"].item()
    del data

    levels = ''.join(str(i) for i in range(1, nlevels + 1))

    if unique:
        np.savez(f'{name_dataset}_n{nlevels}.npz', imgs = imgs_data[:,nlevels-1,:,:,:], metadata = meta_data)

    else:
        np.savez(f'{name_dataset}_n{levels}.npz', imgs = imgs_data[:,0:nlevels,:,:,:], metadata = meta_data)

    print("Finalizado")

In [3]:
save_dataset("h2f_multiresolution_train",5, unique=True)
save_dataset("h2f_multiresolution_val",5, unique=True)
save_dataset("h2f_multiresolution_test",5, unique=True)

Finalizado
Finalizado
Finalizado


In [ ]:
def save_kfolds(train_name,val_name,test_name, diccionario, type_kfold):
    
    train = np.load(f"datasets\{train_name}.npz", allow_pickle=True)
    imgs_train = train["imgs"]
    meta_train = train["metadata"]
    del train
    print("Set de Entrenamiento Cargado")

    val = np.load(f"datasets\{val_name}.npz", allow_pickle=True)
    imgs_val = val["imgs"]
    meta_val = val["metadata"]
    del val
    print("Set de Validacion Cargado")

    test = np.load(f"datasets\{test_name}.npz", allow_pickle=True)
    imgs_test = test["imgs"]
    meta_test = test["metadata"]
    del test
    print("Set de Test Cargado")

    full_imgs = np.concatenate([imgs_train, imgs_val, imgs_test])
    del imgs_train, imgs_val, imgs_test
    print("Concatenacion de imagenes lista")

    df_train = pd.DataFrame(meta_train.item())
    df_val = pd.DataFrame(meta_val.item())
    df_test = pd.DataFrame(meta_test.item())

    full_meta = pd.concat((df_train,df_val,df_test)).reset_index(drop=True)
    del df_train, df_val, df_test
    print("Concatenacion de metadata lista\n")

    print("Descargando KFolds")
    for nfold in range(5):

        train_idxs = diccionario["train_idxs"][nfold]
        test_idxs = diccionario["test_idxs"][nfold]

        np.savez(f'D:\datasets_kfolds\{train_name}_{type_kfold}_kfold{nfold+1}.npz', imgs = full_imgs[train_idxs], metadata = dict(full_meta.iloc[train_idxs]))
        np.savez(f'D:\datasets_kfolds\{test_name}_{type_kfold}_kfold{nfold+1}.npz', imgs = full_imgs[test_idxs], metadata = dict(full_meta.iloc[test_idxs]))

        print(f"KFold N°{nfold+1} completado")

    print("Finalizado")

In [3]:
save_kfolds("h2f_multiresolution_train",
            "h2f_multiresolution_val",
            "h2f_multiresolution_test",
            last_folds,
            "last")

Set de Entrenamiento Cargado
Set de Validacion Cargado
Set de Test Cargado
Concatenacion de imagenes lista
Concatenacion de metadata lista

Descargando KFolds
KFold N°1 completado
Finalizado
